In [0]:
import os
import numpy as np
import pandas as pd
import gc

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/twitter_cleaned_no_punkt.csv') 
train.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'class'], axis=1, inplace=True)
train.tweet.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['tweet', 'mal'])

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42      
X, X_test, y, y_test  = train_test_split(train.tweet, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
!pip install transformers==2.6.0

     |████████████████████████████████| 542kB 3.4MB/s 
     |████████████████████████████████| 3.7MB 51.2MB/s 
     |████████████████████████████████| 870kB 40.2MB/s 
     |████████████████████████████████| 1.0MB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=af5d85c35f06f0cbd58c833a6dfbfd98074d290aba4570ccfb5081a25f652624
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
!pip install tensorflow==2.2.0rc2

In [0]:
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
import traitlets
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from sklearn.metrics import roc_auc_score
from tensorflow.keras import backend as K
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score

warnings.simplefilter("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
#Transformer implementation is based on the following code: https://www.kaggle.com/miklgr500/jigsaw-tpu-bert-with-huggingface-and-keras

In [0]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')

# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer("vocab.txt", lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)

In [0]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [0]:
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #x = tf.keras.layers.Dropout(0.35)(cls_token)
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=3e-5),
                  metrics=[tf.keras.metrics.AUC()])
    
    return model

In [0]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fsc = []

In [0]:
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models/cv_models_twitter'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models/models_twitter'

In [0]:
TRAIN = False

In [0]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    
    # Tokenization within the CV to ensure not having overfitting
    
    X_train = fast_encode(X_train, fast_tokenizer, maxlen=33)
    X_val = fast_encode(X_val, fast_tokenizer, maxlen=33)

    train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .shuffle(42)
    .batch(64)
    .prefetch(AUTO)
    )

    valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_val, y_val))
    .batch(64)
    .cache()
    .prefetch(AUTO)
    )
    
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    with strategy.scope():
      transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
      model = build_model(transformer=transformer_layer,  max_len=33)
    

    model.layers[1].trainable = False
    model.summary()

    print('Fitting')
    
    if TRAIN:
      history              = model.fit(train_dataset,
                                      epochs=1
                                      )
      model.save_weights(f'{cv_models_path}/BERT_last_fold_{c}.h5')
    else:
      model.load_weights(f'{cv_models_path}/BERT_last_fold_{c}.h5')

    probs                = model.predict(valid_dataset, batch_size=64, verbose=1)
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')

    threshold = 0.7
    probs_class = probs.copy()
    probs_class[probs_class >= threshold] = 1 
    probs_class[probs_class < threshold] = 0
    precision = precision_score(y_val, probs_class) 
    recall    = recall_score(y_val, probs_class)
    fscore    = f1_score(y_val, probs_class)

    fsc.append(fscore)
    print(f' fscore {fscore}')
    
    del model
    K.clear_session()
    gc.collect()

 fold 0


INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 33)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 33, 768), (None,  109482240 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 109,483,009
Trainable params: 769
Non-trainable params: 109,482,240
_________________________________________________________________
Fitting


55/55 [==============================] - 4s 75ms/step
 average precision 0.9974521920439117
 roc auc 0.9882499811288985
 fscore 0.9764183185235816


### CV Results

In [0]:
np.array(auc).mean()

0.9972105914908728

In [0]:
np.array(auc).std()

0.0005733760778388841

In [0]:
np.array(roc).mean()

0.9871643310246805

In [0]:
np.array(roc).std()

0.002144604898507007

In [0]:
np.array(fsc).mean()

0.9760424059828301

In [0]:
np.array(fsc).std()

0.0022250888767570897

### Full test

In [0]:
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.104.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
x_train = fast_encode(X, fast_tokenizer, maxlen=33)
x_test = fast_encode(X_test, fast_tokenizer, maxlen=33)

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y))
    .shuffle(42)
    .batch(64)
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test, y_test))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

In [0]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
    model = build_model(transformer_layer, max_len=33)

CPU times: user 11.3 s, sys: 4 s, total: 15.3 s
Wall time: 20.7 s


In [0]:
model.layers[1].trainable = False

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 33)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 33, 768), (None,  109482240 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 109,483,009
Trainable params: 769
Non-trainable params: 109,482,240
_________________________________________________________________


In [0]:
if TRAIN:
  train_history = model.fit(
      train_dataset,
      epochs=1
  )
  model.save_weights(f'{models_path}/BERT_last.h5')
else:
  model.load_weights(f'{models_path}/BERT_last.h5')

275/275 [==============================] - 31s 111ms/step - loss: 0.1649 - auc: 0.9666


In [0]:
probs = model.predict(test_dataset, batch_size=64, verbose=1)



112/112 [==============================] - 3s 24ms/step


In [0]:
threshold = 0.7
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0

In [0]:
auc_f   = average_precision_score(y_test, probs)
roc_f   = roc_auc_score(y_test, probs)
f_score = f1_score(y_test, probs_class)

In [0]:
auc_f

0.9974419485178517

In [0]:
roc_f

0.9877179572338465

In [0]:
f_score

0.9749894022891055